# Study of the powerlaw exponent as a function of 8 micron intensity


In [ ]:
import numpy as np
import astropy.io.fits as pyfits
import pandas as pd

import matplotlib.pyplot as plt

from scipy.stats import pearsonr

In [ ]:
#### User input ####

## information to fetch the necessary data
dir_path = "../data/data_FEEDBACK/" ## path to the data directory
name_CII = "_CII_final_res20_grid8_0p5_clean_integrated" ## identifier of the [CII] data file
name_8m = "_Spitzer_IRAC_8_20_8" ## identifier of the 8 micron data file
plot_path = "../plots/CII_correlations/"

list_regions = [ ## list of the regions to be studied
    "CYGNUSX",
    "M16",
    "M17",
    "NGC7538",
    "RCW36",
    "RCW49",
    "RCW79",
    "RCW120",
    "W40",
    "ORIONA"
]

## Spitzer 8 micron bandwidth
bandwidth_8micron = 1.58e13 ## Hz
conv_OrionA = 2.9*4.8e-5*0.2021/10.4*0.74

## minimal intensity for the [CII] data
min_cut_CII = 20.

## [CII] convert K km/s -> erg/s/cm2/sr
conv_CII = 7.0354e-6

## number of datapoints to be considered to fit a powerlaw exponent
num_data_points = [500, 1000]

####################